In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

from utils.trainer_utils import TrainConfig
from flearn.trainer.fedavg import FedAvg
from flearn.trainer.fesem import FeSEM
from flearn.trainer.fedgroup import FedGroup
from flearn.trainer.ifca import IFCA

def main():
    config = TrainConfig('femnist', 'mclr', 'fedgroup')
    config.group_config['consensus'] = False
    config.trainer_config['dynamic'] = False
    config.trainer_config['swap_p'] = 0
    config.client_config['temperature'] = None
    config.trainer_config['temp_metrics'] = 'cosine'
    config.trainer_config['temp_func'] = 'step'
    trainer = FedGroup(config)
    trainer.train()
    #trainer.train_locally()

main()

Num GPUs Available:  1
The dataset size: 18345, train size: 16421, test size: 1924.
The train client count: 200. The test client count: 200.
Pre-training takes 32.06604599952698s seconds
SVD takes 0.06745624542236328s seconds
EDC Matrix calculation takes 0.0063018798828125s seconds
MADC Matrix calculation takes 0.015819311141967773s seconds
Clustering takes 0.03511309623718262s seconds
Clustering Results: Counter({1: 29, 3: 20, 0: 20, 2: 16, 4: 15})
---------- Round 0 ----------
Round 0, Group 0 has 4 client.
Round 0, Group 1 has 6 client.
Round 0, Group 2 has 4 client.
Round 0, Group 3 has 4 client.
Round 0, Group 4 has 2 client.
groupbase.py Train NKS: [241.0, 257.0, 223.0, 422.0, 216.0]
Round 0, Train ACC: 0.4879,            Train Loss: 1.0759
Round 0, Group: 0, Train ACC: 0.5311,                    Train Loss: 1.0751
Round 0, Group: 3, Train ACC: 0.4475,                    Train Loss: 1.1059
Round 0, Group: 4, Train ACC: 0.5067,                    Train Loss: 1.0552
Round 0, Group:

Round 6, Group 0 has 5 client.
Round 6, Group 1 has 5 client.
Round 6, Group 2 has 6 client.
Round 6, Group 3 has 2 client.
Round 6, Group 4 has 2 client.
groupbase.py Train NKS: [520.0, 293.0, 137.0, 375.0, 395.0]
Round 6, Train ACC: 0.7372,            Train Loss: 0.807
Round 6, Group: 0, Train ACC: 0.7692,                    Train Loss: 0.7535
Round 6, Group: 3, Train ACC: 0.7543,                    Train Loss: 0.7725
Round 6, Group: 4, Train ACC: 0.708,                    Train Loss: 0.8404
Round 6, Group: 2, Train ACC: 0.7653,                    Train Loss: 0.7996
Round 6, Group: 1, Train ACC: 0.6658,                    Train Loss: 0.8985
groupbase.py Test NKS: [481, 345, 207, 354, 329]
Round 6, Test(Partial) ACC: 0.3998,            Test(Partial) Loss: 2.4133
Round 6, Group: 0, Test ACC: 0.6279,                    Test Loss: 1.3736
Round 6, Group: 3, Test ACC: 0.371,                    Test Loss: 3.6143
Round 6, Group: 4, Test ACC: 0.4879,                    Test Loss: 1.6881
Round

groupbase.py Test NKS: [529, 362, 285, 388, 360]
Round 12, Test(Complete) ACC: 0.5338,            Test(Complete) Loss: 2.0162
Round 12, Group: 0, Test ACC: 0.6692,                    Test Loss: 1.4318
Round 12, Group: 3, Test ACC: 0.6519,                    Test Loss: 1.6964
Round 12, Group: 4, Test ACC: 0.5684,                    Test Loss: 2.5317
Round 12, Group: 2, Test ACC: 0.3119,                    Test Loss: 2.0628
Round 12, Group: 1, Test ACC: 0.4278,                    Test Loss: 2.7381
Round: 12, Training time: 4.215, Test time: 0.295,                 Inter-Group Aggregate time: 0
---------- Round 13 ----------


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Reshape, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Reshape Layer
model.add(Reshape((28, 28, 1)))
# Conv Layer
model.add(Conv2D(24, 5, padding='same', activation='relu', input_shape=(28, 28, 1)))
# MaxPool Layer
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
# Flatten Layer
model.add(Flatten())
# Dense Layer
model.add(Dense(256, 'relu'))
# Output Layer
model.add(Dense(10, 'softmax'))

#opt = tf.keras.optimizers.SGD(learning_rate=lr)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
import numpy as np
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Output Layer
model.add(Dense(10, 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    
opt = tf.keras.optimizers.SGD(learning_rate=0.003)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Output Layer
model.add(Dense(10, 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
#opt = tf.keras.optimizers.Adam(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])


In [ ]:
print(model.get_weights())

In [ ]:
tf.random.set_seed(100)
sess.run(model.initializer)
print(model.get_weights())